In [2]:
import os

api_key = os.getenv("openai_api_key")

In [4]:
import aiohttp
from bs4 import BeautifulSoup
import pprint
from playwright.async_api import async_playwright
from langchain.chains import create_extraction_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI

from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
from langchain_community.document_transformers import BeautifulSoupTransformer


In [5]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125", openai_api_key=api_key)

schema1 = {
    "properties": {
        "food": {"type": "string"},
        "price": {"type": "integer"},
        "description": {"type": "string"}
    },
    "required": ["food", "price", "description"]
}
schema2 = {
    "properties": {
        "food category": {"title":"Category", "description": "The main food category in menu", "type": "string"},
        "food item": {"title":"Food name", "description": "The main food item in menu", "type": "string"},
        "food price": {"title": "Price", "description": "The price of each food item", "type": "integer"},
        "description": {"title": "Description", "description": "The description of food item","type": "string"}
    },
    "required": ["food category","food item", "food price", "description"]
}

structured_schema = {
    "properties": {
        "product": {"type": "string"},
        "description": {"type": "string"},
    },
    "required": ["product", "description"],
}

def extract(content: str, schema: dict):
    return create_extraction_chain(schema=schema, llm=llm).run(content)

c:\Users\shres\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter
def scrape_with_playwright(urls, schema):
    loader = AsyncHtmlLoader(urls)
    docs = loader.load()
    
    html2text = Html2TextTransformer()
    docs_transformed = html2text.transform_documents(docs)
    
    print("Extracting content with LLM")

    # Grab the first 1000 tokens of the site
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=800, chunk_overlap=0
    )
    splits = splitter.split_documents(docs_transformed)

    # Process the first split
    extracted_content = extract(schema=schema, content=splits[0].page_content)
    pprint.pprint(extracted_content)
    return extracted_content

urls = ["https://cafebrazil.com/menu/"]
extracted_content = scrape_with_playwright(urls, schema=schema2)


#######################################################################

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125", openai_api_key=api_key)

In [56]:
urls = ["https://getglasskin.com/","https://cafebrazil.com/menu/"]
url = ["https://cafebrazil.com/menu", "https://thetownedeli.com/food-menu/"]
url2 = ["https://cafebrazil.com"]

from langchain_community.document_loaders import SeleniumURLLoader
loader = SeleniumURLLoader(urls=url2)
data = loader.load()
# data1 = data[1]
# data2 = data
pprint.pprint(data)

[Document(page_content='Chickem & Waffles\n\nChicken\n\nWaffles\n\nOnline Ordering for Carry Out is HERE\n\nPancake Tacos\n\nPancake\n\nTacos\n\nOnline Ordering for Carry Out is HERE\n\nBreakfast Relleno\n\nBreakfast\n\nRelleno\n\nOnline Ordering for Carry Out is HERE\n\nSlide 1\n\nSPOTLIGHT: SANDWICHES & SALADS\n\nSouthwest Vegan Burger\n\nHomemade Vegan Burger with a side of Sweet Potato Fries and Fresh Guacamole . . .\n\nView The Menu\n\nThis is my favorite dish. The food was AMAZING!!!!\n\nDaphne K.\n\nDallas, TX\n\nSlide 1\n\nSPOTLIGHT: PANCAKES\n\nCrepes, Pancakes & More\n\nThree Large Pancakes Served With Syrup and Butter\n\nView The Menu\n\nI am 100% sure that the collaboration with Nextend was the best choice we could have made. Nextend turned a highly complex product into a simple but premium user experience.\n\nTrienke van Aartsen\n\nAmper AI\n\nAll Dishes Are Made With The Best Ingredients\n\nShowcase 1\n\nShowcase 3\n\nShowcase 5\n\nShowcase 6', metadata={'source': 'https:

In [69]:
#Alternative loader
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

url = "https://cafebrazil.com"
loader = RecursiveUrlLoader(
    url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()


Unable to load from https://cafebrazil.com/menu/signature-specials-cb-favorites/. Received error HTTPSConnectionPool(host='www.cafebrazil.com', port=443): Max retries exceeded with url: /menu (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001215E266900>: Failed to resolve 'www.cafebrazil.com' ([Errno 11001] getaddrinfo failed)")) of type ConnectionError
c:\Users\shres\AppData\Local\Programs\Python\Python312\Lib\html\parser.py:171: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [71]:
pprint.pprint(docs)

[Document(page_content="\n\n\n\n\n\n\nWelcome To Cafe Brazil - Cafe Brazil\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Skip to primary navigation Skip to main contentFacebookInstagramPinterestTwitter\nCafe BrazilDallas' Best Coffee House and Restaurant | Late Night Dining | Free Wi-FiORDER NOW\nLocations\nMenu\n\nKid’s Menu\nCatering\n\n\nCB Store\nCB Scoop\n\nAbout Us\nCafe Brazil Awards\nCareers\nCatering\nSpecial Offers\nOur Blog\n\n\nCB Perks Club\nContact\n\n\n\n\n\n\n \nChickem & WafflesChicken &Waffles\n\n\nOnline Ordering for Carry Out is HEREPancake TacosPancakeTacos\n\n\nOnline Ordering for Carry Out is HEREBreakfast RellenoBreakfastRelleno\n\n\nOnline Ordering for Carry Out is HERE \n\n\n\n\n\n\n\n\n \nSlide 1SPOTLIGHT: SANDWICHES & SALADSSouthwest Vegan BurgerHomemade Vegan Burger with a side of Sweet Potato Fries and Fresh Guacamole . . .View The MenuThis is my

In [57]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class food menu analyzer. Your job is to extract all menu link from the given {data}"),
    ("user", "{input}")
])

In [81]:

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class food menu analyzer. Your job is to extract menu data from the given {docs[0]}"),
    ("user", "{input}")
])

In [73]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Menu_Links(BaseModel):
    url_link: str = Field(description="url link of the menu information")
    
class Menu(BaseModel):
    category: str = Field(description="category of the food menu")
    menu_item: str = Field(description="menu item name")
    description: str = Field(description="menu item description with ingredients")
    price: float = Field(description="price of the menu in USD. 'NA' if not available")

parser = JsonOutputParser(pydantic_object=Menu)
# parser = JsonOutputParser(pydantic_object=Menu_Links)

In [76]:
chain1 = prompt | llm | parser

In [82]:
results = chain1.invoke({"input": "Please extract all the url links of the information related to the menu from the given {docs} and store it in dictionary",
                         "data": docs})

KeyError: "Input to ChatPromptTemplate is missing variables {'docs'}.  Expected: ['docs', 'input'] Received: ['input', 'data']"

In [64]:
pprint.pprint(results)

{'menu_links': ['https://cafebrazil.com',
                'https://cafebrazil.com',
                'https://cafebrazil.com']}


In [54]:
chain = prompt | llm | parser

In [75]:
results = chain1.invoke({"input": "Please extract the category, list of all the menu items name within that category along with description in single string format and price of the menu item from {docs} in a json format. If price is not available, please leave the field empty",
              "data": docs})

KeyError: "Input to ChatPromptTemplate is missing variables {'docs'}.  Expected: ['docs', 'input'] Received: ['input', 'data']"

In [42]:
pprint.pprint(results)

{'menu': [{'category': 'Breakfast',
           'items': [{'description': '',
                      'name': 'Chicken & Waffles',
                      'price': ''},
                     {'description': '', 'name': 'Pancake Tacos', 'price': ''},
                     {'description': '',
                      'name': 'Breakfast Relleno',
                      'price': ''}]},
          {'category': 'SPOTLIGHT: SANDWICHES & SALADS',
           'items': [{'description': 'Homemade Vegan Burger with a side of '
                                     'Sweet Potato Fries and Fresh Guacamole',
                      'name': 'Southwest Vegan Burger',
                      'price': ''}]},
          {'category': 'SPOTLIGHT: PANCAKES',
           'items': [{'description': 'Three Large Pancakes Served With Syrup '
                                     'and Butter',
                      'name': 'Crepes, Pancakes & More',
                      'price': ''}]}]}


In [13]:
pprint.pprint(results)

{'menu': [{'category': 'Signature Specials',
           'items': [{'description': 'Homemade Waffles topped with hand-cut '
                                     'Chicken Tenders & Bacon, served with '
                                     'Hash Brown Casserole and topped with an '
                                     'Egg Over Easy (Make it Cholula spicy '
                                     '.99)',
                      'name': 'CHICKEN, BACON & WAFFLES',
                      'price': '16.99'},
                     {'description': 'Two Eggs, a Chorizo Empanada topped with '
                                     'Homemade Spicy Cream Sauce, Cheddar, and '
                                     'Tomatoes, served with Rosemary Potatoes '
                                     'and Grilled Homemade White Bread',
                      'name': 'CB CHORIZO EMPANADA BREAKFAST',
                      'price': '13.59'},
                     {'description': 'Scrambled Eggs, Fajita Steak, Bell '
     

###############################################################

In [32]:
urls = ["https://getglasskin.com/","https://cafebrazil.com"]
url1 = ["https://cafebrazil.com/menu/"]
url_2 = ["https://getglasskin.com/"]
from langchain_community.document_loaders import SeleniumURLLoader
loader = SeleniumURLLoader(urls=url1)
data = loader.load()
pprint.pprint(data)

[Document(page_content='Menu\n\nSignature Specials | Omelets  |  Breakfast |  Appetizers |  Tacos |  Sandwiches |  Sides & Beverages  | Seasonal Menu  | Desserts\n\nSIGNATURE SPECIALS\n\nTry one of our signature specials.\n\nCHICKEN, BACON & WAFFLES\n\n16.99\n\nHomemade Waffles topped with hand-cut Chicken Tenders & Bacon, served with Hash Brown Casserole and topped with an Egg Over Easy (Make it Cholula spicy .99)\n\nCB CHORIZO EMPANADA BREAKFAST\n\n13.59\n\nTwo Eggs, a Chorizo Empanada topped with Homemade Spicy Cream Sauce, Cheddar, and Tomatoes, served with Rosemary Potatoes and Grilled Homemade White Bread\n\nCHIPOTLE STEAK & EGGS\n\n14.99\n\nScrambled Eggs, Fajita Steak, Bell Peppers, Mushrooms and Onions, topped with Chipotle Sauce and Monterey Jack, served with Rosemary Potatoes, and Two Flour Tortillas\n\nBREAKFAST RELLENO\n\n13.59\n\nRoasted Poblano Pepper filled with Scrambled Eggs, Chorizo and Cheddar, topped with Homemade Spicy Cream Sauce and served with Fried Sweet Potat

In [23]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


#building the llm model
llm_model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125", openai_api_key=openai_key_2)



In [24]:
#Defining output datatypes within the class and initialize the parser
class Menu(BaseModel):
    menu: list = Field(description="Python list of dictionaries containing category of food, food menu, price of the menu in USD. 'NA' if not available, menu description with ingredients")
    
#Initialize a parser
parser = PydanticOutputParser(pydantic_object=Menu)

In [35]:
human_prompt = HumanMessagePromptTemplate.from_template("{request}\n{format_instructions}")
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

request = chat_prompt.format_prompt(
    request= "Give me all the details about the menu from the {data}",
    format_instructions = parser.get_format_instructions()
).to_messages()

In [36]:
results = llm_model(request, temperature=0)
result_values = parser.parse(results.content) #menu class object
print(result_values)

menu=[{'category': 'Appetizers', 'food_menu': 'Garlic Parmesan Wings', 'price': 9.99, 'description': 'Crispy chicken wings tossed in garlic parmesan sauce'}, {'category': 'Entrees', 'food_menu': 'Grilled Salmon', 'price': 15.99, 'description': 'Fresh grilled salmon served with lemon butter sauce'}, {'category': 'Desserts', 'food_menu': 'Chocolate Lava Cake', 'price': 6.99, 'description': 'Warm chocolate cake with a gooey chocolate center'}]


In [6]:
import pandas as pd

df = pd.read_csv("Google_all_details.csv") 
# res_url = df['Website'].tolist()

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (447528796.py, line 3)